In [1]:
import sys
sys.executable

'c:\\Users\\mkongathi\\Ramu\\Ppn\\Codes\\Projects\\ML Algorithms\\Financial Market Analysis Dashboard\\myenv\\Scripts\\python.exe'

In [2]:
from bokeh.plotting import figure, show, save, output_file, output_notebook
from bokeh.layouts import column, row, gridplot
from bokeh.models import HoverTool, Select, CustomJS, ColumnDataSource, DateRangeSlider
from bokeh.palettes import Category20, Set3
from bokeh.io import curdoc
from bokeh.transform import factor_cmap
import pandas as pd
import numpy as np
import sys
sys.path.append('../src')

from data_collection import FinancialDataCollector
from data_processing import FinancialDataProcessor

In [3]:
# Load data
collector = FinancialDataCollector()
stock_data = collector.load_stock_data()
processor = FinancialDataProcessor(stock_data)

output_notebook()

Data loaded from data\raw\stock_data.pkl


Loading BokehJS ...

In [ ]:
# Plot-1. Interactive Stock Price Chart
def create_bokeh_stock_chart():
    
    ticker = 'AAPL'             # Prepare data
    df = stock_data[ticker]['prices'].reset_index()
    df['Date'] = pd.to_datetime(df['Date'])
    
    source = ColumnDataSource(df)
    
    p = figure(x_axis_type='datetime',          # Create figure
               width=900, height=400,
               title=f"{ticker} Stock Price with Technical Indicators",
               toolbar_location="above")
    
    inc = df['Close'] > df['Open']              # Add candlestick-like visualization using segments and rects
    dec = df['Open'] > df['Close']
    
    p.segment(df.Date[inc], df.High[inc], df.Date[inc], df.Low[inc], color="green", line_width=1)       # High-low lines
    p.segment(df.Date[dec], df.High[dec], df.Date[dec], df.Low[dec], color="red", line_width=1)
    
    w = 12*60*60*1000               # half day in ms
    p.rect(df.Date[inc], (df.Open[inc] + df.Close[inc])/2, w, abs(df.Close[inc] - df.Open[inc]),        # Open-close rectangles
           fill_color="green", line_color="green", alpha=0.8)
    p.rect(df.Date[dec], (df.Open[dec] + df.Close[dec])/2, w, abs(df.Close[dec] - df.Open[dec]),
           fill_color="red", line_color="red", alpha=0.8)
    
    p.line(df.Date, df.MA_20, legend_label="MA 20", line_width=2, color="blue", alpha=0.8)          # Add moving averages
    p.line(df.Date, df.MA_50, legend_label="MA 50", line_width=2, color="orange", alpha=0.8)
    
    hover = HoverTool(                  # Add hover tool
        tooltips=[
            ('Date', '@Date{%F}'),
            ('Open', '@Open{$0.00}'),
            ('High', '@High{$0.00}'),
            ('Low', '@Low{$0.00}'),
            ('Close', '@Close{$0.00}'),
            ('Volume', '@Volume{0,0}')
        ],
        formatters={'@Date': 'datetime'}
    )
    p.add_tools(hover)
    
    # Styling
    p.legend.location = "top_left"
    p.legend.click_policy = "hide"
    
    output_file("./static/bokeh_stock_chart.html")
    save(p)

    # show(p)
    
    return p

stock_chart = create_bokeh_stock_chart()

In [9]:
# Plot-2. Multi-Stock Comparison Dashboard
def create_multi_stock_dashboard():
    stocks = ['AAPL', 'GOOGL', 'MSFT', 'TSLA']
    colors = Category20[len(stocks)]
    
    normalized_data = {}                # Normalize prices to start at 100
    for stock in stocks:
        prices = stock_data[stock]['prices']['Close']
        normalized_data[stock] = (prices / prices.iloc[0]) * 100
    
    df = pd.DataFrame(normalized_data)
    df.reset_index(inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    
    source = ColumnDataSource(df)
    
    p = figure(x_axis_type='datetime',              # Create figure
               width=900, height=500,
               title="Normalized Stock Price Comparison (Base=100)",
               toolbar_location="above")
    
    for i, stock in enumerate(stocks):              # Add lines for each stock
        p.line(x='Date', y=stock, source=source, 
               legend_label=stock, line_width=2, 
               color=colors[i], alpha=0.8)
        
        p.circle(x='Date', y=stock, source=source,      # Add circle markers
                size=3, color=colors[i], alpha=0.6)
    
    hover = HoverTool(                                  # Add hover tool
        tooltips=[('Date', '@Date{%F}'), ('Price Index', '$y{0.00}')],
        formatters={'@Date': 'datetime'}
    )
    p.add_tools(hover)
    
    p.legend.location = "top_left"
    p.legend.click_policy = "hide"
    p.yaxis.axis_label = "Normalized Price (Base=100)"
    p.xaxis.axis_label = "Date"
    
    output_file("./static/bokeh_multi_stock.html")
    save(p)
    # show(p)
    
    return p

multi_stock_chart = create_multi_stock_dashboard()

In [13]:
# Plot-3. Risk-Return Scatter Plot
def create_risk_return_bokeh():
    
    risk_return_data = []           # Calculate risk-return metrics
    
    for ticker in list(stock_data.keys())[:20]:
        returns = stock_data[ticker]['prices']['Returns'].dropna()
        annual_return = returns.mean() * 252 * 100
        annual_volatility = returns.std() * np.sqrt(252) * 100
        market_cap = stock_data[ticker]['info'].get('marketCap', 0) / 1e9
        
        risk_return_data.append({
            'ticker': ticker,
            'return': annual_return,
            'risk': annual_volatility,
            'market_cap': market_cap
        })
    
    df = pd.DataFrame(risk_return_data)
    source = ColumnDataSource(df)
    
    p = figure(width=800, height=600,               # Create figure
               title="Risk vs Return Analysis",
               toolbar_location="above")
    
    source.data['scaled_size'] = np.sqrt(source.data['market_cap']) * 2

    scatter = p.circle(x='risk', y='return', size='scaled_size',         # Add scatter plot with size based on market cap
                      source=source, alpha=0.7, color='blue')
    
    from bokeh.models import LabelSet                       # Add labels
    labels = LabelSet(x='risk', y='return', text='ticker',
                     x_offset=5, y_offset=5, source=source,
                     text_font_size="9pt")
    p.add_layout(labels)
    
    hover = HoverTool(          # Add hover tool
        tooltips=[
            ('Stock', '@ticker'),
            ('Annual Return', '@return{0.2f}%'),
            ('Annual Risk', '@risk{0.2f}%'),
            ('Market Cap', '@market_cap{$0.0f}B')
        ]
    )
    p.add_tools(hover)
    
    # Add reference lines
    p.line([0, df['risk'].max()], [0, 0], line_dash="dashed", alpha=0.5, color="black")
    p.line([df['risk'].mean(), df['risk'].mean()], 
           [df['return'].min(), df['return'].max()], 
           line_dash="dashed", alpha=0.5, color="red")
    
    p.xaxis.axis_label = "Annual Volatility (%)"
    p.yaxis.axis_label = "Annual Return (%)"
    
    output_file("./static/bokeh_risk_return.html")
    save(p)
    # show(p)
    
    return p

risk_return_chart = create_risk_return_bokeh()

In [14]:
# Plot-4. Volume Analysis
def create_volume_analysis_bokeh():
    ticker = 'AAPL'
    df = stock_data[ticker]['prices'].reset_index()
    df['Date'] = pd.to_datetime(df['Date'])
    df['Color'] = ['green' if c >= o else 'red' for c, o in zip(df['Close'], df['Open'])]
    
    source = ColumnDataSource(df)
    
    # Create subplots
    p1 = figure(x_axis_type='datetime', width=900, height=300,
                title=f"{ticker} - Price Analysis",
                toolbar_location="above")
    
    p2 = figure(x_axis_type='datetime', width=900, height=200,
                title="Volume Analysis",
                x_range=p1.x_range,  # Link x-axes
                toolbar_location="above")
    
    # Price plot
    p1.line('Date', 'Close', source=source, line_width=2, color='blue', alpha=0.8)
    p1.yaxis.axis_label = "Price ($)"
    
    # Volume plot
    p2.vbar(x='Date', top='Volume', source=source, width=86400000, color='Color', alpha=0.7)
    p2.yaxis.axis_label = "Volume"
    p2.xaxis.axis_label = "Date"
    
    # Add hover tools
    hover1 = HoverTool(tooltips=[('Date', '@Date{%F}'), ('Close', '@Close{$0.00}')])
    hover2 = HoverTool(tooltips=[('Date', '@Date{%F}'), ('Volume', '@Volume{0,0}')])
    p1.add_tools(hover1)
    p2.add_tools(hover2)
    
    # Combine plots
    layout = column(p1, p2)
    
    output_file("./static/bokeh_volume_analysis.html")
    save(layout)
    # show(layout)
    
    return layout

volume_analysis = create_volume_analysis_bokeh()

In [16]:
# plot 5. Sector Performance Dashboard
def create_sector_performance_bokeh():
    from config import SECTOR_MAPPING
    
    # Calculate sector performance
    sector_performance = []
    
    for sector, tickers in SECTOR_MAPPING.items():
        sector_returns = []
        for ticker in tickers:
            if ticker in stock_data:
                returns = stock_data[ticker]['prices']['Returns'].dropna()
                annual_return = returns.mean() * 252 * 100
                sector_returns.append(annual_return)
        
        if sector_returns:
            avg_return = np.mean(sector_returns)
            volatility = np.std(sector_returns)
            sector_performance.append({
                'sector': sector,
                'return': avg_return,
                'volatility': volatility,
                'count': len(sector_returns)
            })
    
    df = pd.DataFrame(sector_performance)
    source = ColumnDataSource(df)
    
    # Create bar chart
    p = figure(x_range=df['sector'], width=800, height=500,
               title="Sector Performance Comparison",
               toolbar_location="above")
    
    # Color mapping
    color_mapper = factor_cmap(field_name='sector', palette=Set3[len(df)], factors=df['sector'].unique())

    p.vbar(x='sector', top='return', source=source, width=0.8,
           color=color_mapper, alpha=0.8)
    
    # Add hover tool
    hover = HoverTool(
        tooltips=[
            ('Sector', '@sector'),
            ('Avg Return', '@return{0.2f}%'),
            ('Volatility', '@volatility{0.2f}%'),
            ('Stock Count', '@count')
        ]
    )
    p.add_tools(hover)
    
    p.xgrid.grid_line_color = None
    p.xaxis.major_label_orientation = 45
    p.yaxis.axis_label = "Average Annual Return (%)"
    p.xaxis.axis_label = "Sector"
    
    output_file("./static/bokeh_sector_performance.html")
    save(p)
    # show(p)
    
    return p

sector_performance_chart = create_sector_performance_bokeh()